In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import tensorflow as tf
import numpy as np

from utils import reset
from caps import layers
from yolophem import utils, losses

In [28]:
IMG_SIZE = 448
S = 7
B = 3

In [34]:
reset()

predictions = tf.placeholder(shape=[None, S, S, B, 4], dtype=tf.float32, name="predictions")
confidences = tf.placeholder(shape=[None, S, S, B], dtype=tf.float32, name='confidences')
labels_long = tf.placeholder(shape=[None, 5], dtype=tf.int64, name="labels")

In [38]:
losses.yolophem_loss(predictions, confidences, labels_long, IMG_SIZE)

<tf.Tensor 'yolophem_loss_3/add_1:0' shape=() dtype=float32>

In [4]:
sample_idx, labels = tf.split(labels_long, [1,4], axis=1)

labels_centered = utils.center(labels, name='labels_centered')
labels_localized, cell_idx = utils.localize(labels_centered, IMG_SIZE, S, name='labels_localized')

predictions_selected = tf.gather_nd(predictions, tf.concat([sample_idx, cell_idx], axis=-1), name='predictions_selected')
labels_localized_tiled = tf.tile(tf.reshape(labels_localized, [-1, 1, 4]), multiples=[1, B, 1], name='targets_localized_tiled')

prediction_ious = utils.intersection_over_union(predictions_selected, labels_localized_tiled)
predictor_idx = tf.reshape(tf.argmax(prediction_ious, axis=-1), [-1, 1])

targets_idx = tf.concat([sample_idx, cell_idx, predictor_idx], axis=-1)

batch_size = tf.shape(predictions)[0]
targets_shape = tf.stack([tf.cast(batch_size, tf.int64), S, S, B, 4])
mask_shape = tf.stack([tf.cast(batch_size, tf.int64), S, S, B, 1])

targets = tf.scatter_nd(targets_idx, labels_localized, shape=targets_shape)
responsibility_mask = tf.scatter_nd(targets_idx, tf.ones_like(sample_idx, dtype=tf.float32), shape=mask_shape)

responsible_predictions = predictions * responsibility_mask
box_losses = (responsible_predictions - targets) ** 2
box_loss = tf.reduce_sum(box_losses)

In [352]:
responsibility_mask_reshaped =tf.reshape(responsibility_mask, [-1, S, S, B])

responsible_confidence_losses = responsibility_mask_reshaped * (confidences - utils.intersection_over_union(predictions, targets))
irresponsible_confidence_losses = (1 - responsibility_mask_reshaped) * confidences

In [344]:
box_loss

<tf.Tensor 'Sum:0' shape=() dtype=float32>

In [331]:
tf.scatter_nd(targets_idx, labels_localized, shape=shape)

<tf.Tensor 'ScatterNd:0' shape=(?, 5, 5, 3, 4) dtype=float32>

In [188]:
tf.gather_nd(predictions, tf.concat([sample_idx, cell_idx], axis=-1))

<tf.Tensor 'GatherNd:0' shape=(?, 3, 4) dtype=float32>

In [14]:
labels_np = np.array(
    [
        [0, 220, 110, 40, 80],
        [0, 10, 19, 110, 70],
        [1, 11, 119, 20, 20],
        [1, 11, 123, 21, 21],
        [2, 400, 400, 10, 10]
    ]
)

In [193]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    results = sess.run(
        [test],
        feed_dict={
            predictions: np.zeros([3, S, S, B, 4]),
            labels_long: labels_np
        }
    )
    
    for result in results:
        print(result)

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [235]:
box1 = np.array([[0,0,3,3]]).reshape([1,1,1,4])
box2 = np.array([[2,2,3,3]]).reshape([1,1,1,4])

In [249]:
reset()

a = tf.constant(box1)
b = tf.constant(box2)

a_center = utils.center(a)
b_center = utils.center(b)


In [250]:
res = utils.intersection_over_union(a_center,b_center )

In [251]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    results = sess.run(
        [res],
    )
    
    for result in results:
        print(result)

[[[0.05882353]]]


In [252]:
1/17

0.058823529411764705